# Clustering Crypto

In [ ]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [ ]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [ ]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [ ]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)


### Data Preprocessing

In [ ]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
columns=["CoinName", "Algorithm", "IsTrading", "ProofType", "TotalCoinsMined", "TotalCoinSupply"]
crypto_df.columns=columns


In [ ]:
# Keep only cryptocurrencies that are on trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)

In [ ]:
# Keep only cryptocurrencies with a working algorithm


In [ ]:
# Remove the "IsTrading" column
initial_df = crypto_df.drop(['IsTrading'], axis=1)
initial_df.head()


In [ ]:
# Remove rows with at least 1 null value
second_df = first_df.dropna()

In [ ]:
# Remove rows with cryptocurrencies having no coins mined
third_df = second_df[second_df!=0].dropna()

In [ ]:
# Drop rows where there are 'N/A' text values
fourth_df = fourth_df[fourth_df!=NaN].dropna()

In [ ]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df


In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
fifth_df = fourth_df.drop(columns='CoinName', axis=1)
fifth_df.head()

In [ ]:
# Create dummy variables for text features
dummies = pd.get_dummies(fourth_df[['Algorithm', 'ProofType']], drop_first=True)
X = pd.concat([fourth_df.drop(['Algorithm', 'ProofType'],axis=1), dummies],axis=1)
X.head()

In [ ]:
# Standardize data
data_scaler = StandardScaler()
data_scaler.fit_transform(X)

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X)

In [ ]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=X.index
)
pcs_df.head()

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
model.fit(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["class"] = model.labels_
pcs_df.head()

In [ ]:
clustered_df = pd.concat([third_df, pcs_df], axis=1)
clustered_df

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 3",
    y="PC 2",
    z="PC 1",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], width=800)

In [ ]:
# Print the total number of tradable cryptocurrencies
count= clustered_df['CoinName'].count()
print(f'There are {count} tradable cryptocurrencies')

In [ ]:
new_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
new_df

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
data_scaler2 = StandardScaler()
data_scaler.fit_transform(new_df)

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
new_df.hvplot.scatter(y="TotalCoinsMined", x="TotalCoinSupply")